# Assignment 4: Decision Tree Learning

In this assignment, you will work with a class of reinforcement learning agents called decision trees to attempt to classify features according to some decision boundary.


This assignment is due on T-Square on November 3 by 9:35 AM.

Introduction:
-------

For this assignment we're going to need an explicit way to make structured decisions. The following is a decision node- a class representing some atomic choice in a binary decision graph. It can represent a class label (i.e. a final decision) or a binary decision to guide the us through a flow-chart to arrive at a decision.

In [28]:
class DecisionNode():

    def __init__(self, left, right, decision_function,class_label=None):
        self.left = left
        self.right = right
        self.decision_function = decision_function
        self.class_label = class_label

    def decide(self, feature):
        if self.class_label != None:
            return self.class_label
        
        return self.left.decide(feature) if self.decision_function(feature) else self.right.decide(feature)

Part 1: Warmup: Building a tree by hand
--------
20 pts.

In the below code block, construct a tree of decision nodes by hand in order to classify the data below. Select tests to be as small as possible (in terms of attributes), breaking ties among tests with the same number of attributes by selecting the one that classifies the greatest number of examples correctly. If multiple tests have the same number of attributes and classify the same number of examples, then break the tie using attributes with lower index numbers (e.g. select $A_1$ over $A_2$)

| Datum  | $A_1$ | $A_2$ | $A_3$ | $A_4$ |  y  |
| -------| :---: | :---: | :---: | :---: | ---:|
| $x_1$  |   1   |   0   |   0   |   0   |  1  |
| $x_2$  |   1   |   0   |   1   |   1   |  1  |
| $x_3$  |   0   |   1   |   0   |   0   |  1  |
| $x_4$  |   0   |   1   |   1   |   0   |  0  |
| $x_5$  |   1   |   1   |   0   |   1   |  1  |
| $x_6$  |   0   |   1   |   0   |   1   |  0  |
| $x_7$  |   0   |   0   |   1   |   1   |  1  |
| $x_8$  |   0   |   0   |   1   |   0   |  0  |


In [29]:
examples = [[1,0,0,0],
            [1,0,1,1],
            [0,1,0,0],
            [0,1,1,0],
            [1,1,0,1],
            [0,1,0,1],
            [0,0,1,1],
            [0,0,1,0]]

classes = [1,1,1,0,1,0,1,0]

def return0(feature):
    return feature[0]
def return1(feature):
    return feature[1]
def return2(feature):
    return feature[2]
def return3(feature):
    return feature[3]

A4_1 = DecisionNode(DecisionNode(None,None,None,1),DecisionNode(None,None,None,0),return3)
A4_2 = DecisionNode(DecisionNode(None,None,None,0),DecisionNode(None,None,None,1),return3)

A3 = DecisionNode(DecisionNode(None,None,None,0),A4_2,return2)
A2 = DecisionNode(A3,A4_1,return1)
A1 = DecisionNode(DecisionNode(None,None,None,1),A2,return0)
# Constructing nodes one at a time,
# build a decision tree as specified above.
# There exists a correct tree with less than 6 nodes.

decision_tree_root = A1

Part 1b: Validation
--------

Now that we have a decision tree, we're going to need some way to evaluate its performance. In most cases we'd reserve a portion of the training data for evaluation, or use cross validation, bot for now let's just see how your tree does on the provided examples. In the stubbed out code below, fill out the methods to compute accuracy, precision, recall, and the confusion matrix for your classifier output.

In [30]:
def confusion_matrix(classifier_output, true_labels):
    #TODO output should be [[true_positive, false_negative], [false_positive, true_negative]]
    true_positive = 0
    false_negative = 0
    false_positive = 0
    true_negative = 0
    
    for i in range(8):
        if classifier_output[i] == true_labels[i]:
            if true_labels[i] == 0:
                true_negative+=1
            else:
                true_positive+=1
        else:
            if true_labels[i] == 0:
                false_positive+=1
            else:
                false_negative+=1

    return [[true_positive,false_negative], [false_positive, true_negative]]
    raise NotImplemented()

def precision(classifier_output, true_labels):
    #TODO precision is measured as: true_positive/ (true_positive + false_positive)
    matrix = confusion_matrix(classifier_output, true_labels)
    return matrix[0][0]/(matrix[0][0] + matrix[1][0])
    raise NotImplemented()
    
def recall(classifier_output, true_labels):
    #TODO: recall is measured as: true_positive/ (true_positive + false_negative)
    matrix = confusion_matrix(classifier_output, true_labels)
    return matrix[0][0]/(matrix[0][0] + matrix[0][1])
    raise NotImplemented()
    
def accuracy(classifier_output, true_labels):
    #TODO accuracy is measured as:  correct_classifications / total_number_examples
    matrix = confusion_matrix(classifier_output, true_labels)
    return (matrix[0][0] + matrix[1][1])/8
    raise NotImplemented()
    
classifier_output = [decision_tree_root.decide(example) for example in examples]

# Make sure your hand-built tree is 100% accurate.
p1_accuracy = accuracy( classifier_output, classes )
p1_precision = precision(classifier_output, classes)
p1_recall = recall(classifier_output, classes)
p1_confusion_matrix = confusion_matrix(classifier_output, classes)

Part 2: Decision Tree Learning
-------
40 pts.

As the number of examples we have grows, it rapidly becomes impractical to build these trees by hand, so it becomes necessary to specify a procedure by which we can automagically construct these trees.

For starters, let's consider the following algorithm (a variation of C4.5) for the construction of a decision tree from a given set of examples:

    1) Check for base cases: 
         a)If all elements of a list are of the same class, return a leaf node with the appropriate class label.
         b)If a specified depth limit is reached, return a leaf labeled with the most frequent class.

    2) For each attribute alpha: evaluate the normalized information gain gained by splitting on alpha

    3) Let alpha_best be the attribute with the highest normalized information gain

    4) Create a decision node that splits on alpha_best

    5) Recur on the sublists obtained by splitting on alpha_best, and add those nodes as children of node

In the \_\_build_tree\__ method below implement the above algorithm. In the "classify" method below, write a function to produce classifications for a list of features once your decision tree has been build.

In [26]:
def entropy(class_vector):
    # TODO: Compute the Shannon entropy for a vector of classes
    # Note: Classes will be given as either a 0 or a 1.
    raise NotImplemented()
    
def information_gain(previous_classes, current_classes ):
    # TODO: Implement information gain
    raise NotImplemented()

class DecisionTree():

    def __init__(self, depth_limit=float('inf')):
        self.root = None
        self.depth_limit = depth_limit

    def fit(self, features, classes):
        self.root = self.__build_tree__(features, classes)

    def __build_tree__(self, features, classes, depth=0):   
        #TODO Implement the algorithm as specified above
        raise NotImplemented()
        
    def classify(self, features):
        #TODO Use a fitted tree to classify a list of feature vectors
        # Your output should be a list of class labels (either 0 or 1)
        raise NotImplemented()


Part 2b: Validation
--------

For this part of the assignment we're going to use a relatively simple dataset (banknote authentication, found in 'part_2_data.csv'. In the section below there are methods to load the data in a consistent format.

In general, reserving part of your data as a test set can lead to unpredictable performance- a serendipitous choice of your train or test split could give you a very inaccurate idea of how your classifier performs. That's where k-fold cross validation comes in.

In the below method, we'll split the dataset at random into k equal subsections, then iterating on each of our k samples, we'll reserve that sample for testing and use the other k-1 for training. Averaging the results of each fold should give us a more consistent idea of how the classifier is doing.


In [31]:
def load_csv(data_file_path, class_index=-1):
    handle = open(data_file_path, 'r')
    contents = handle.read()
    handle.close()
    rows = contents.split('\n')
    out = [  [float(i) for i in r.split(',')] for r in rows if r ]
    classes= map(int,  out[:, class_index])
    features = out[:, :class_index]
    return features, classes

def generate_k_folds(dataset, k):
    #TODO this method should return a list of folds,
    # where each fold is a tuple like (training_set, test_set)
    # where each set is a tuple like (examples, classes)
    raise NotImplemented()

dataset = load_csv('part2_data.csv')
ten_folds = generate_k_folds(dataset, 10)

#on average your accuracy should be higher than 60%.
accuracies = []
precisions = []
recalls = []
confusion = []

for fold in ten_folds:
    train, test = fold
    train_features, train_classes = train
    test_features, test_classes = test
    tree = DecisionTree( )
    tree.fit( train_features, train_classes)
    output = tree.classify(test_features)
    
    accuracies.append( accuracy(output, test_classes))
    precisions.append( precision(output, test_classes))
    recalls.append( recall(output, test_classes))
    confusion.append( confusion_matrix(output, test_classes))
    


TypeError: list indices must be integers, not tuple

Part 3: Random Forests
-------
30 pts.

The decision boundaries drawn by decision trees are very sharp, and fitting a decision tree of unbounded depth to a list of examples almost inevitably leads to overfitting. In an attempt to decrease the variance of our classifier we're going to use a technique called 'Bootstrap Aggregating' (often abbreviated 'bagging').

A Random Forest is a collection of decision trees, built as follows:

1) For every tree we're going to build:

    a) Subsample the examples provided us (with replacement) in accordance with a provided example subsampling rate.
    
    b) From the sample in a), choose attributes at random to learn on (in accordance with a provided attribute subsampling rate)
    
    c) Fit a decision tree to the subsample of data we've chosen (to a certain depth)
    
Classification for a random forest is then done by taking a majority vote of the classifications yielded by each tree in the forest after it classifies an example.



In [1]:
class RandomForest():

    def __init__(self, num_trees, depth_limit, example_subsample_rate, attr_subsample_rate):
        self.trees = []
        self.num_trees = num_trees
        self.depth_limit = depth_limit
        self.example_subsample_rate = example_subsample_rate
        self.attr_subsample_rate = attr_subsample_rate

    def fit(self, features, classes):
        # TODO implement the above algorithm to build a random forest of decision trees
        raise NotImplemented()

    def classify(self, features):
        # TODO implement classification for a random forest.
        raise NotImplemented()

#TODO: As with the DecisionTree, evaluate the performance of your RandomForest on the dataset for part 2.
# on average your accuracy should be higher than 75%.

#  Optimize the parameters of your random forest for accuracy for a forest of 5 trees.
# (We'll verify these by training one of your RandomForest instances using these parameters
#  and checking the resulting accuracy)

#  Fill out the function below to reflect your answer:

def ideal_parameters():
    raise NotImplemented()
    return ideal_depth_limit, ideal_esr, ideal_asr


Part 4: Challenge!
-------
10 pts

You've been provided with a sample of data from a research dataset in 'challenge_data.pickle'. It is serialized as a tuple of (features, classes). I have reserved a part of the dataset for testing. The classifier that performs most accurately on the holdout set wins (so optimize for accuracy). To get full points for this part of the assignment, you'll need to get at least an average accuracy of 80% on the data you have, and at least an average accuracy of 60% on the holdout set.

Ties will be broken by submission time.

First place:  +3% on your final grade

Second place: +2% on your final grade

Third place:  +1% on your final grade


In [ ]:
class ChallengeClassifier():
    
    def __init__(self):
        # initialize whatever parameters you may need here-
        # this method will be called without parameters 
        # so if you add any to make parameter sweeps easier, provide defaults
        raise NotImplemented()
        
    def fit(self, features, classes):
        # fit your model to the provided features
        raise NotImplemented()
        
    def classify(self, features):
        # classify each feature in features as either 0 or 1.
        raise NotImplemented()